In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from skimage import io
from torchvision import transforms
from models import BaseModel

In [ ]:
model_name = 'model-epoch-100.ckpt'

In [ ]:
num_classes = 6
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
checkpoint = torch.load('./models/' + model_name)
model = BaseModel(num_classes)
model = model.to(device)
model.load_state_dict(checkpoint['state_dict'])
model.eval();

In [ ]:
input_dir = './dataset/faces_images'
df = pd.read_csv('./dataset/test_vision.csv')

data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])])

In [ ]:
df_output = pd.DataFrame(columns=['prediction'])

In [ ]:
for i in df.index:
    image_path = input_dir + '/' + df['filename'][i]
    image = io.imread(image_path)
    image = data_transforms(image)
    image = image.unsqueeze(0).to(device)
    output = model(image)  # [batch_size, num_classes]
    _, pred = torch.max(output, dim=1)
    df_output = df_output.append({'prediction': pred.item() + 1}, ignore_index=True)

In [ ]:
df_output.to_csv('sample_output_vision.csv', index=False)